In [1]:
import polars as pl
from tqdm import tqdm
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy

## Load file comment đã được gán nhãn

In [2]:
df = pd.read_csv("comment_labeled.csv")
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_28836\3167720376.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("comment_labeled.csv")


,id,user_id,text,resource_id,create_time,sentiment_label
0,Cm_61,105,Hello teacher,V_13690,1.565886e+12,1.0
1,Cm_74,9520252,The function of this evaluation is too hidden,V_286363,1.566560e+12,0.0
2,Cm_171,11731,is_teacher_join,V_454872,1.567700e+12,1.0
3,Cm_177,11731,Socrates Aristotle Plato,V_454872,1.567700e+12,1.0
4,Cm_180,6875014,test,V_586172,1.567701e+12,1.0


## Bỏ đi những cột không cần thiết

In [3]:
df.drop(columns=["user_id", "text", "resource_id", "create_time"], inplace=True)
df.rename(columns={"id": "comment_id"}, inplace=True)
df.head()

,comment_id,sentiment_label
0,Cm_61,1.0
1,Cm_74,0.0
2,Cm_171,1.0
3,Cm_177,1.0
4,Cm_180,1.0


## Load file course-comment.txt (các comment ứng với khóa học tương ứng)

In [4]:
course_comment = pd.read_csv("course-comment.txt", sep='\t', header=None, names=['course_id', 'comment_id'])
course_comment.head()

,course_id,comment_id
0,C_597284,Cm_4
1,C_597334,Cm_61
2,C_682246,Cm_74
3,C_697602,Cm_83
4,C_605983,Cm_88


## Nối hai file dựa trên comment_id chung

In [9]:
joined_df = pd.merge(course_comment, df, on="comment_id")
joined_df.drop(columns=["comment_id"], inplace=True)
joined_df.head()

,course_id,sentiment_label
0,C_597334,1.0
1,C_682246,0.0
2,C_696700,1.0
3,C_696700,1.0
4,C_734023,1.0


## Đếm số comment positive, negative, neutral và tổng số comment

In [13]:
joined_df['positive'] = joined_df['sentiment_label'].apply(lambda x: 1 if x == 2.0 else 0)
joined_df['negative'] = joined_df['sentiment_label'].apply(lambda x: 1 if x == 0.0 else 0)
joined_df['neutral'] = joined_df['sentiment_label'].apply(lambda x: 1 if x == 1.0 else 0)
joined_df.drop(columns=["sentiment_label"], inplace=True)
joined_df = joined_df.groupby("course_id").sum()
joined_df['total_cmt'] = joined_df['positive'] + joined_df['negative'] + joined_df['neutral']
joined_df.head()

,positive,negative,neutral,total_cmt
course_id,,,,
C_1017355,176,18,364,558
C_1017419,77,0,19,96
C_1025064,15,1,75,91
C_1025076,1,0,0,1
C_1025079,35,8,56,99


In [14]:
joined_df = joined_df.reset_index()
joined_df.head()

,course_id,positive,negative,neutral,total_cmt
0,C_1017355,176,18,364,558
1,C_1017419,77,0,19,96
2,C_1025064,15,1,75,91
3,C_1025076,1,0,0,1
4,C_1025079,35,8,56,99


## Load file reply đã được gán nhãn

In [17]:
df = pl.read_csv("reply_labeled.csv")
df.head()

id,user_id,text,create_time,sentiment_label
str,str,str,i64,i64
"""Rp_1""","""U_10030806""","""test reply""",1565009754000,1
"""Rp_2""","""U_10031397""","""praise""",1565368746000,2
"""Rp_3""","""U_10031531""","""I like it very much""",1565476775000,2
"""Rp_4""","""U_10031508""","""You are great too""",1565621014000,2
"""Rp_5""","""U_10031508""","""Yeah, right""",1565621091000,1


In [18]:
df = df.to_pandas()
df.head()

,id,user_id,text,create_time,sentiment_label
0,Rp_1,U_10030806,test reply,1565009754000,1
1,Rp_2,U_10031397,praise,1565368746000,2
2,Rp_3,U_10031531,I like it very much,1565476775000,2
3,Rp_4,U_10031508,You are great too,1565621014000,2
4,Rp_5,U_10031508,"Yeah, right",1565621091000,1


## Bỏ đi những cột không cần thiết

In [20]:
df = df.drop(columns=["user_id", "text", "create_time"])
df.rename(columns={"id": "reply_id"}, inplace=True)
df.head()

,reply_id,sentiment_label
0,Rp_1,1
1,Rp_2,2
2,Rp_3,2
3,Rp_4,2
4,Rp_5,1


## Load file course_reply.csv (các reply ứng với khóa học tương ứng)

In [21]:
course_reply = pd.read_csv("course_reply.csv")
course_reply.head()

,course_id,reply_id
0,C_696700,Rp_15
1,C_696700,Rp_13
2,C_696700,Rp_14
3,C_734023,Rp_359
4,C_734023,Rp_360


## Nối hai file dựa trên reply_id chung

In [22]:
joined_df_reply = pd.merge(course_reply, df, on="reply_id")
joined_df_reply.drop(columns=["reply_id"], inplace=True)
joined_df_reply.head()

,course_id,sentiment_label
0,C_696700,1
1,C_696700,1
2,C_696700,1
3,C_734023,1
4,C_734023,1


## Đếm số reply positive, negative, neutral và tổng số reply

In [23]:
joined_df_reply['positive'] = joined_df_reply['sentiment_label'].apply(lambda x: 1 if x == 2.0 else 0)
joined_df_reply['negative'] = joined_df_reply['sentiment_label'].apply(lambda x: 1 if x == 0.0 else 0)
joined_df_reply['neutral'] = joined_df_reply['sentiment_label'].apply(lambda x: 1 if x == 1.0 else 0)
joined_df_reply.drop(columns=["sentiment_label"], inplace=True)
joined_df_reply = joined_df_reply.groupby("course_id").sum()
joined_df_reply['total_reply'] = joined_df_reply['positive'] + joined_df_reply['negative'] + joined_df_reply['neutral']
joined_df_reply.head()

,positive,negative,neutral,total_reply
course_id,,,,
C_1011639,0,0,5,5
C_1017355,44,21,115,180
C_1017419,0,0,1,1
C_1025064,1,0,0,1
C_1025076,2,0,0,2


In [24]:
joined_df_reply = joined_df_reply.reset_index()
joined_df_reply.head()

,course_id,positive,negative,neutral,total_reply
0,C_1011639,0,0,5,5
1,C_1017355,44,21,115,180
2,C_1017419,0,0,1,1
3,C_1025064,1,0,0,1
4,C_1025076,2,0,0,2


## Nối hai file đã được tính ở trên dựa trên course_id chung, có một số trường hợp bất thường có reply nhưng lại không có comment (có thể do bộ dataset MOOCCubeX thống kê không đủ).

In [25]:
final_df = pd.merge(joined_df, joined_df_reply, on="course_id", how="outer")
final_df.head()

,course_id,positive_x,negative_x,neutral_x,total_cmt,positive_y,negative_y,neutral_y,total_reply
0,C_1011639,NaN,NaN,NaN,NaN,0.0,0.0,5.0,5.0
1,C_1017355,176.0,18.0,364.0,558.0,44.0,21.0,115.0,180.0
2,C_1017419,77.0,0.0,19.0,96.0,0.0,0.0,1.0,1.0
3,C_1025064,15.0,1.0,75.0,91.0,1.0,0.0,0.0,1.0
4,C_1025076,1.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0


## Các reply cũng là bản chất là comment nên trong trường hợp này những giá trị NaN sẽ được gán giá trị 0 và cộng lại.

In [26]:
final_df.fillna(0, inplace=True)
final_df['positive'] = final_df['positive_x'] + final_df['positive_y']
final_df['negative'] = final_df['negative_x'] + final_df['negative_y']
final_df['neutral'] = final_df['neutral_x'] + final_df['neutral_y']
final_df['total_cmt'] = final_df['total_cmt'] + final_df['total_reply']
final_df.drop(columns=['positive_x', 'negative_x', 'neutral_x', 'total_reply', 'positive_y', 'negative_y', 'neutral_y'], inplace=True)
final_df.head()

,course_id,total_cmt,positive,negative,neutral
0,C_1011639,5.0,0.0,0.0,5.0
1,C_1017355,738.0,220.0,39.0,479.0
2,C_1017419,97.0,77.0,0.0,20.0
3,C_1025064,92.0,16.0,1.0,75.0
4,C_1025076,3.0,3.0,0.0,0.0


In [27]:
final_df.to_csv("label_comment.csv", index=False)